In [1]:
!pip install transformers[ja,sentencepiece,torch]

In [2]:
from transformers import pipeline

## 文書分類

In [3]:
from torch import positive
text_classification_pipeline = pipeline(model='llm-book/bert-base-japanese-v3-marc_ja')
positive_text = '世界には言葉がわからなくても感動する音楽がある'
print(text_classification_pipeline(positive_text)[0])

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


{'label': 'positive', 'score': 0.9994138479232788}


In [4]:
negative_text = '世界には言葉が出ないほどひどい音楽がある'
print(text_classification_pipeline(negative_text)[0])

{'label': 'negative', 'score': 0.9901888966560364}


## 自然言語推論

In [5]:
nli_pipeline = pipeline(model='llm-book/bert-base-japanese-v3-jnli')
text = '二人の男性がジェット機を見ています'
entailment_text = 'ジェット機を見ている人が二人います'
print(nli_pipeline({'text':text, 'text_pair':entailment_text}))

{'label': 'entailment', 'score': 0.9964311122894287}


In [6]:
contradiction_text = '二人の男性が飛んでいます'
print(nli_pipeline({'text':text, 'text_pair':contradiction_text}))

{'label': 'contradiction', 'score': 0.9990535378456116}


In [7]:
neural_text = '二人の男性が、白い飛行機を眺めています'
print(nli_pipeline({'text':text, 'text_pair':neural_text}))

{'label': 'neutral', 'score': 0.9579214453697205}


## 意味的類似度計算

In [8]:
text_sim_pipeline = pipeline(
    model='llm-book/bert-base-japanese-v3-jsts',
    function_to_apply='none'
)
text = '川べりでサーフボードを持った人たちがいます'
sim_text = 'サーファーたちが川べりに立っています'
result = text_sim_pipeline({'text':text, 'text_pair':sim_text})
print(result['score'])

3.5703558921813965


In [9]:
dissim_text = 'トイレの壁に黒いタオルがかけられています'
result = text_sim_pipeline({'text':text, 'text_pair':dissim_text})
print(result['score'])

0.04162175580859184


In [10]:
from torch.nn.functional import cosine_similarity

sim_enc_pipeline = pipeline(
    model='llm-book/bert-base-japanese-v3-unsup-simcse-jawiki',
    task='feature-extraction'
)
text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb = sim_enc_pipeline(sim_text, return_tensors=True)[0][0]
sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())

0.8568589687347412


In [11]:
dissim_emb = sim_enc_pipeline(dissim_text, return_tensors=True)[0][0]
dissim_pair_score = cosine_similarity(text_emb, dissim_emb, dim=0)
print(dissim_pair_score.item())

0.45887047052383423


## 固有表現認識

In [12]:
from pprint import pprint

ner_pipeline = pipeline(
    model='llm-book/bert-base-japanese-v3-ner-wikipedia-dataset',
    aggregation_strategy='simple'
)
text = '大谷翔平は岩手県水沢市出身のプロ野球選手'
pprint(ner_pipeline(text))

[{'end': None,
  'entity_group': '人名',
  'score': 0.99823624,
  'start': None,
  'word': '大谷 翔平'},
 {'end': None,
  'entity_group': '地名',
  'score': 0.9986874,
  'start': None,
  'word': '岩手 県 水沢 市'}]


## 要約生成

In [13]:
text2text_pipeline = pipeline(
    model='llm-book/t5-base-long-livedoor-news-corpus'
)
article = "ついに始まった3連休。テレビを見ながら過ごしている人も多いのではないだろうか？今夜オススメなのは何といっても、NHKスペシャル「世界を変えた男スティーブ・ジョブズ」だ。実は知らない人も多いジョブズ氏の養子に出された生い立ちや、アップル社から一時追放されるなどの経験。そして、彼が追い求めた理想の未来とはなんだったのか、ファンならずとも気になる内容になっている。今年、亡くなったジョブズ氏の伝記は日本でもベストセラーになっている。今後もアップル製品だけでなく、世界のジョブズ氏の影響は大きいだろうと想像される。ジョブズ氏のことをあまり知らないという人もこの機会にぜひチェックしてみよう。世界を変えた男　スティーブジョブズ（NHKスペシャル）"
print(text2text_pipeline(article)[0]['generated_text'])

今夜はNHKスペシャル「世界を変えた男スティーブ・ジョブズ」


## transformersの基本的な使い方

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('abeja/gpt2-large-japanese')
tokenizer.tokenize('今日が天気が良いので')

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


['▁', '今日', 'が', '天気', 'が良い', 'の', 'で']

In [15]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained('abeja/gpt2-large-japanese')
inputs = tokenizer('今日は天気が良いので', return_tensors='pt')
outputs = model.generate(**inputs, max_length=15, pad_token_id=tokenizer.pad_token_id)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

今日は天気が良いので外でお弁当を食べました。
